In [3]:
# ==========================================
# 3_train_letter_model.ipynb
# ==========================================
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pickle
from sklearn.model_selection import train_test_split
# =========================================================================
# 1️⃣ Load dataset processed + Augmentasi (Menggunakan Augmentasi Lebih Baik)
# =========================================================================
X, y = [], []
data_dir = "../../dataset/processed_letters"
# Definisikan parameter augmentasi
NOISE_RANGE = 0.03 # Sedikit ditingkatkan
AUG_FACTOR = 3     # Melipatgandakan data 3 kali
for label in os.listdir(data_dir):
    folder = os.path.join(data_dir, label)
    if not os.path.isdir(folder):
        continue
    for file in os.listdir(folder):
        if file.endswith(".npy"):
            data = np.load(os.path.join(folder, file), allow_pickle=True)
            if len(data) == 0: continue

            # 1. Sampel asli
            X.append(data)
            y.append(label)

            # 2. Augmentasi N kali
            for _ in range(AUG_FACTOR):
                # Augmentasi: noise Gaussian untuk distribusi yang lebih alami
                noise = np.random.normal(0, NOISE_RANGE, data.shape) 
                data_aug = data + noise
                X.append(data_aug)
                y.append(label)
X = np.array(X, dtype='float32')
y = np.array(y)
print(f"✅ Total samples (termasuk Augmentasi): {len(X)}")
print(f"📂 Total classes: {len(set(y))}")
# ==========================================
# 2️⃣ Encode label & 3️⃣ Normalisasi + Reshape
# ==========================================
le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))
num_classes = y_encoded.shape[1]
# Gunakan StandardScaler untuk normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Reshape untuk 1D-CNN: (samples, features, 1)
X_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1) 
print(f"✅ X shape after reshape for CNN: {X_cnn.shape}")
print(f"✅ y shape: {y_encoded.shape}")
# ==========================================
# 4️⃣ Split dataset
# ==========================================
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn, y_encoded, test_size=0.2, random_state=42, stratify=y
)
print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
# ==========================================
# 5️⃣ Buat Model 1D-CNN (Arsitektur Diperkaya)
# ==========================================
input_shape = (X_cnn.shape[1], 1)
model = Sequential([
    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=256, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=512, kernel_size=3, activation='relu'),
    GlobalAveragePooling1D(),
    Dropout(0.4),

    Dense(256, activation='relu'),
    Dropout(0.4),

    Dense(num_classes, activation='softmax')
])
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
# ==========================================
# 6️⃣ Training dengan Advanced Callbacks
# ==========================================
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=0.00001, verbose=1)
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_split=0.15,
    callbacks=[es, reduce_lr],
    shuffle=True
)
# ==========================================
# 7️⃣ Simpan model, label encoder, scaler
# ==========================================
os.makedirs("../../models", exist_ok=True)
model.save("../../models/sign_letter_model.keras")
with open("../../models/label_letter_encoder.pkl", "wb") as f:
    pickle.dump(le, f)
with open("../../models/scaler_letter.pkl", "wb") as f:
    pickle.dump(scaler, f)
print("✅ Model 1D-CNN huruf berhasil disimpan di folder '../../models/'")

✅ Total samples (termasuk Augmentasi): 22368
📂 Total classes: 26
✅ X shape after reshape for CNN: (22368, 225, 1)
✅ y shape: (22368, 26)
Train size: 17894, Test size: 4474


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 223, 128)          512       
                                                                 
 batch_normalization (Batch  (None, 223, 128)          512       
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 111, 128)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 109, 256)          98560     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 256)          1024      
 chNormalization)                                                
                                                        

2025-10-14 01:32:43.560867: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


238/238 [==============================] - 7s 24ms/step - loss: 4.3948 - accuracy: 0.0464 - val_loss: 4.0369 - val_accuracy: 0.0566 - lr: 0.0010
Epoch 2/200
238/238 [==============================] - 5s 22ms/step - loss: 4.2678 - accuracy: 0.0510 - val_loss: 3.6463 - val_accuracy: 0.0596 - lr: 0.0010
Epoch 3/200
238/238 [==============================] - 5s 21ms/step - loss: 3.9677 - accuracy: 0.0522 - val_loss: 3.3760 - val_accuracy: 0.0592 - lr: 0.0010
Epoch 4/200
238/238 [==============================] - 5s 21ms/step - loss: 3.7531 - accuracy: 0.0538 - val_loss: 3.3773 - val_accuracy: 0.0577 - lr: 0.0010
Epoch 5/200
238/238 [==============================] - 5s 21ms/step - loss: 3.6081 - accuracy: 0.0571 - val_loss: 3.4118 - val_accuracy: 0.0641 - lr: 0.0010
Epoch 6/200
238/238 [==============================] - 5s 21ms/step - loss: 3.5135 - accuracy: 0.0614 - val_loss: 3.5276 - val_accuracy: 0.0544 - lr: 0.0010
Epoch 7/200
238/238 [==============================] - 5s 22ms/step - 